In [1]:
!pip install -q spaces
!pip install -q gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.0.240 requires pydantic<2,>=1, but you have pydantic 2.7.1 which is incompatible.
langchain-core 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-community 0.2.0 requires langchain<0.3.0,>=0.2.0, but you have langchain 0.0.240 which is incompatible.
langchain-community 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.


In [2]:
!pip install -q transformers==4.40.0
!pip install -q datasets
!pip install -q pillow
!pip install -q numpy
!pip install -q torch
!pip install -q langchain
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q langchain-community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-community 0.2.0 requires langchain<0.3.0,>=0.2.0, but you have langchain 0.0.240 which is incompatible.
langchain-community 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
gradio 4.31.4 requires pydantic>=2.0, but you have pydantic 1.10.15 which is incompatible.


In [3]:
!pip install -q PyPDF2

In [4]:
!pip install -qU \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-community 0.2.0 requires langchain<0.3.0,>=0.2.0, but you have langchain 0.0.240 which is incompatible.
langchain-community 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.


In [5]:
!pip install -q pypdf

In [6]:
!pip install pinecone-client

In [7]:
from pypdf import PdfReader
import pandas as pd

In [8]:
#reader = PdfReader('Baseline_Criteria.pdf')#Morocco-Travel-Guide-DMA.pdf
reader = PdfReader('Morocco-Travel-Guide-DMA.pdf')#Morocco-Travel-Guide-DMA.pdf

print(f"Number of pages in the pdf file : {len(reader.pages)}")

text = ''

for page in reader.pages:
  text += page.extract_text()

Number of pages in the pdf file : 53


In [9]:
# Split the text into chunks
chunks = [text[i:i+500] for i in range(0, len(text), 500)]

In [10]:
# Create a DataFrame with the extracted chunks
df = pd.DataFrame(chunks, columns=['text'])

df['index'] = range(1, len(df) + 1)

df = df[['index', 'text']]

df

,index,text
0,1,\nMOROCCO TRAVEL GUIDE BY DESERT MOROCCO \nAD...
1,2,cceptance of th e “No Liability” policy. If yo...
2,3,not blame them for wanting to travel to this m...
3,4,place where we would be going to in order for...
4,5,the rich history of Morocco. We have also list...
...,...,...
288,289,low and remain proactive for the duration of ...
289,290,Walk purposely and always re member to \navoid...
290,291,o kn ow how to deal with them. Don’t let your ...
291,292,ary where you can relax while fulfilling yo ur...


In [11]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [12]:
from langchain.vectorstores import pinecone

In [13]:
import os
import pinecone

# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or '095388d7-49c5-44e2-a0c3-109dd8154c53',
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
)

In [14]:
print(pinecone.list_indexes())

['index']


In [15]:
index_name = 'index'

# List the existing indexes
existing_indexes = pinecone.list_indexes()

# Check if the index_name exists
if index_name in existing_indexes:
    print(f"Index '{index_name}' already exists. Using the existing index.")
else:
    print(f"Index '{index_name}' does not exist.")
    # If you need to create a new index, you would do it here, but since we're focusing on using an existing one, we'll not create it.

# Proceed with operations using the existing index
# For example, to get the index information
index_info = pinecone.describe_index(index_name)
print(index_info)

Index 'index' already exists. Using the existing index.
IndexDescription(name='index', metric='cosine', replicas=1, dimension=384.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


In [16]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.05131,
 'namespaces': {'': {'vector_count': 5131}},
 'total_vector_count': 5131}

In [17]:
batch_size = 32

for i in range(0, len(df), batch_size):
    i_end = min(len(df), i+batch_size)
    batch = df.iloc[i:i_end]
    ids = [f"{x['index']}" for i, x in batch.iterrows()]
    texts = [x['text'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['text']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

In [18]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.05131,
 'namespaces': {'': {'vector_count': 5131}},
 'total_vector_count': 5131}

In [19]:
from transformers import AutoProcessor, AutoModelForVision2Seq

In [20]:
# # DEVICE = "auto"

processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b-base")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceM4/idefics2-8b-base",
)

/home/user/miniconda/lib/python3.9/site-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/user/miniconda/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/miniconda/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


Model loaded on cuda:0


In [21]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/home/user/miniconda/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [22]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1, # without this output begins repeating
    do_sample=False
)

In [23]:
res = generate_text("Can you tell me something about the history of Rabat the capital of Morocco?")
print(res[0]["generated_text"])

/home/user/miniconda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user/miniconda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Can you tell me something about the history of Rabat the capital of Morocco?
Rabat is a city in northwestern Morocco, located on the Atlantic Ocean at the mouth of the river Bou Regreg. On 10 June 2015, Rabat was named as Capital of Islamic Culture for 2013 by the Islamic Educational, Scientific and Cultural Organization (ISESCO).
The city is divided into two parts: the modern part called Ville Nouvelle, built during the French protectorate from 1912 until the country's independence in 1956, and the old medina with its Kasbah of the Udayas. The city is also known for its role in the Trans-Saharan slave trade.
Rabat has been inhabited since prehistoric times. It was founded by the Phoenicians who called it Sala Colonia. In the 4th century BC, it became an important Roman port under the name of Sala. After being conquered by the Vandals in 441 AD, it was recaptured by the Romans in 473 AD. In 789 AD, Idris I made Rabat his capital after founding the first Muslim state in North Africa.
In

In [24]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [25]:
llm(prompt="Can you tell me something about the history of Rabat the capital of Morocco?")

"\nRabat is a city in northwestern Morocco, located on the Atlantic Ocean at the mouth of the river Bou Regreg. On 10 June 2015, Rabat was named as Capital of Islamic Culture for 2013 by the Islamic Educational, Scientific and Cultural Organization (ISESCO).\nThe city is divided into two parts: the modern part called Ville Nouvelle, built during the French protectorate from 1912 until the country's independence in 1956, and the old medina with its Kasbah of the Udayas. The city is also known for its role in the Trans-Saharan slave trade.\nRabat has been inhabited since prehistoric times. It was founded by the Phoenicians who called it Sala Colonia. In the 4th century BC, it became an important Roman port under the name of Sala. After being conquered by the Vandals in 441 AD, it was recaptured by the Romans in 473 AD. In 789 AD, Idris I made Rabat his capital after founding the first Muslim state in North Africa.\nIn 1033 AD, Almoravid ruler Yusuf ibn Tashfin established a new capital a

In [26]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

In [31]:
query = 'where is Rabat located in Morocco?'

vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='BY DESERT MOROCCO \nADVENTURE 50 \n Enjoy the capital of Morocco: Rabat \nRabat is known to be the capital of Morocco and here, you will never be disappoi nted. You will be able to \nenjoy everything. Also, you will have the chance to see the clean neighborhood o f the area which will \nmake your stroll more fun and fresh. You don’t have to worry about your safet y when you are in Rabat \nbecause the place is safe to everyone and can be navigated easily.  \nExperience the signature dish of Morocco – th', metadata={}),
 Document(page_content='s and the huge gates offer a magnificent view of the culture and trad ition that the \ncity is famous for several years.  \nBe amazed by the historical sites and large monuments found in the city and make  your travel memorable \nand worthwhile.  \nRabat  \nConsidered as Morocco’s top tourist destination, Rabat was named as one of the top travel d estinations \nby CNN in the year 2013. Travelers would love to see the outdoor

In [29]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [30]:
rag_pipeline('Can you tell me something about the history of Rabat the capital of Morocco?')

{'query': 'Can you tell me something about the history of Rabat the capital of Morocco?',
 'result': '\n\nperiod, the city was inhabited by Berbers who were nomadic people. They lived in the area for thousands of years before the Phoenicians arrived. The Phoenicians were the first civilization to settle in the area. They established a port called Chellah. The Phoenicians were followed by the Carthaginians who also settled in the area. The Carthaginians were followed by the Romans who established their own settlement in the area. The Romans were followed by the Vandals who established their own settlement in the area. The Vandals were followed by the Byzantines who established their own settlement in the area. The Byzantines were followed by the Umayyads who established their own settlement in the area. The Umayyads were followed by the Abbasids who established their own settlement in the area. The Abbasids were followed by the Fatimids who established their own settlement in the area. 